In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\Applicativi personali\\Test LDA,keybert,bertopic\\Outputs\\"
filename = "Articles_DF.json"

In [3]:
df = pd.read_json(path+filename,lines=True)

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from collections import Counter
import matplotlib.pyplot as plt

# Preprocessing text

In [5]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Tiziano
[nltk_data]     Pacifico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Tiziano
[nltk_data]     Pacifico\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess_text(text):
    doc = nlp(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [7]:
df['Articoli preprocessati'] = df['Articolo'].apply(preprocess_text)

In [14]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

In [15]:
from nltk.tokenize import word_tokenize

In [16]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

In [21]:
df.head()

,Articolo,Argomento,Descrizione,Articoli preprocessati,Articolo_tokens
0,"Pop culture, short for popular culture, refers...",The Influence of Pop Culture on Society,Explore how popular culture trends influence f...,pop culture short popular culture refer collec...,"[pop, culture, short, popular, culture, refer,..."
1,"In our fast-paced and hectic lives, we often f...",The Benefits of Mindful Eating,Learn how practicing mindful eating can improv...,fast pace hectic life find rush meal barely ta...,"[fast, pace, hectic, life, find, rush, meal, b..."
2,Financial literacy is a crucial skill that eve...,The Importance of Financial Literacy,Learn about the significance of understanding ...,financial literacy crucial skill possess refer...,"[financial, literacy, crucial, skill, possess,..."
3,Fast fashion has become a dominant force in th...,The Impact of Fast Fashion on the Environment,Explore the environmental consequences of fast...,fast fashion dominant force clothing industry ...,"[fast, fashion, dominant, force, clothing, ind..."
4,Physical exercise has long been known to have ...,The Benefits of Regular Exercise for Mental He...,Learn how physical activity can improve mental...,physical exercise long know numerous benefit p...,"[physical, exercise, long, know, numerous, ben..."


# Preparazione per LDA

In [17]:
df['Articolo_tokens'] = df['Articoli preprocessati'].apply(tokenize_text)

In [18]:
dictionary = Dictionary(df['Articolo_tokens'])
corpus = [dictionary.doc2bow(text) for text in df['Articolo_tokens']]

In [22]:
from gensim.models import CoherenceModel

In [122]:
coherence_list = []
for i in range(50,100):
    lda_model = LdaModel(corpus, num_topics=i, id2word=dictionary, passes=20)
    perplexity = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=df['Articolo_tokens'], dictionary=dictionary, coherence='u_mass')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"Numero di topic: {i} -- Perplexity: {perplexity} -- coherence: {coherence_lda}")

Numero di topic: 50 -- Perplexity: -6.497904411234184 -- coherence: -1.8800947160206405
Numero di topic: 51 -- Perplexity: -6.514393832877373 -- coherence: -1.6503354264066876
Numero di topic: 52 -- Perplexity: -6.483584242264504 -- coherence: -1.343597367253222
Numero di topic: 53 -- Perplexity: -6.4880426378636225 -- coherence: -1.8665981112291117
Numero di topic: 54 -- Perplexity: -6.473018763758936 -- coherence: -1.9984422543490405
Numero di topic: 55 -- Perplexity: -6.490004651062538 -- coherence: -2.0980427610969286
Numero di topic: 56 -- Perplexity: -6.4969233344210595 -- coherence: -1.5586563899027455
Numero di topic: 57 -- Perplexity: -6.51927615441179 -- coherence: -1.6447313405564186
Numero di topic: 58 -- Perplexity: -6.527326547057778 -- coherence: -2.1582472238711103
Numero di topic: 59 -- Perplexity: -6.505912263235901 -- coherence: -1.9790884889347797
Numero di topic: 60 -- Perplexity: -6.499612311797471 -- coherence: -1.8741600860864331
Numero di topic: 61 -- Perplexit

KeyboardInterrupt: 

In [ ]:
# Associo ogni id del topic alla lista delle parole chiave
def match_topic_id_with_keywords(topic_id):
    for topic in topics:
        if topic[0]==topic_id:
            return extract_words_from_topic(topic)
    print("no match found")
    
    def match_topic_id_lda_out(topic_id):
    for topic in topics:
        if topic[0]==topic_id:
            return extract_out_from_topic(topic)
    print("no match found")

In [190]:
lda_model = LdaModel(corpus, num_topics=58, id2word=dictionary, passes=20)


In [ ]:
df['LDA keyword'] = df['LDA topic'].apply(match_topic_id_with_keywords)

In [ ]:
df['LDA output'] = df['LDA topic'].apply(match_topic_id_lda_out)

In [199]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\Applicativi personali\\Test LDA,keybert,bertopic\\Outputs\\"
filename = "LDA_results.json"

In [200]:
df.to_json(path+filename, orient="records", lines=True)

In [201]:
df['LDA output'].iloc[0]

'0.048*"influence" + 0.044*"celebrity" + 0.026*"trend" + 0.025*"society" + 0.024*"fashion" + 0.024*"opinion" + 0.022*"shape" + 0.021*"social" + 0.020*"public" + 0.018*"impact"'